А теперь мы поработаем с геоданными.

Возьмём California Housing Dataset, это набор данных для прогнозирования цен на жильё. Давайте импортируем его из scikit-learn.

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

In [ ]:
df = fetch_california_housing(as_frame=True).data
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


Нас интересуют два последних столбца Latitude и Longitude, широта и долгота.

Для работы с ними мы используем библиотеку GeoPandas, который объединяет в себе Pandas и Shapely, библиотеку для геопространственных расчётов.

Давайте установим его.

In [ ]:
!pip install geopandas > _

In [ ]:
import geopandas as gpd

А теперь создадим GeoDataFrame.

In [ ]:
gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']),
        crs=4326
    ).to_crs(epsg=3857)
gdf.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,geometry
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,POINT (-13606581.360 4562487.679)
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,POINT (-13605468.165 4559667.342)
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,POINT (-13607694.555 4558257.461)


Как видите он особо ничем не отличается от обычного DataFrame, единственное отличие в столбце geometry, который представляет собой объект GeoSeries, для которого дополнительно доступны свои атрибуты и методы.

In [ ]:
type(gdf['geometry'])

geopandas.geoseries.GeoSeries

А внутри его лежат объекты Point из библиотеки shapely.

In [ ]:
type(gdf['geometry'][0])

shapely.geometry.point.Point

К слову в GeoPandas можно работать не только с точками, но и с кривыми линиями, полигонами и смесью всего сразу.

Подробнее см.: https://shapely.readthedocs.io/en/stable/manual.html

**County feature**

Теперь давайте генерировать фичи. Калифорния довольно большая, 58 округов. У каждого свой уровень жизни, свой климат, соответственно и свои цены на жильё. Кажется, что категориальная фича о принадлежности округу могла бы помочь.

Для начала нам нужно получить координаты округов. Возьмём их из банка открытых данных Калифорнии https://data.ca.gov (если хотите посетить сайт, то сейчас, на начало 2023 года, нужен VPN).

In [ ]:
import requests, zipfile, io
county_fname = 'https://data.ca.gov/dataset/e212e397-1277-4df3-8c22-40721b095f33/resource/'+\
                       'b0007416-a325-4777-9295-368ea6b710e6/download/ca-county-boundaries.zip'
r2 = requests.get(county_fname)
z2 = zipfile.ZipFile(io.BytesIO(r2.content))
z2.extractall("./ca")

In [ ]:
ca_counties=gpd.read_file('./ca/CA_Counties').to_crs("EPSG:3857")
ca_counties.head(3)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694587,23299110,+39.5769252,-120.5219926,"POLYGON ((-13431319.751 4821511.426, -13431312..."
1,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2499183617,76073827,+38.4500114,-121.3404409,"POLYGON ((-13490651.476 4680831.603, -13490511..."
2,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7084000598,2729814515,+34.5370572,-120.0399729,"MULTIPOLYGON (((-13423116.772 4042044.149, -13..."


Теперь добавим фичи.

In [ ]:
gdf = gpd.overlay(gdf, ca_counties[['NAME', 'geometry']], how='intersection')

In [ ]:
gdf.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,NAME,geometry
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,Alameda,POINT (-13606581.360 4562487.679)
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,Alameda,POINT (-13605468.165 4559667.342)
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,Alameda,POINT (-13607694.555 4558257.461)


Появился столбец NAME.

In [ ]:
from collections import Counter
Counter(gdf['NAME'])

Counter({'Alameda': 1017,
         'Contra Costa': 597,
         'Alpine': 3,
         'Amador': 28,
         'Butte': 156,
         'Calaveras': 32,
         'Colusa': 16,
         'Solano': 199,
         'Del Norte': 16,
         'El Dorado': 120,
         'Fresno': 525,
         'Tulare': 258,
         'Glenn': 26,
         'Humboldt': 127,
         'Imperial': 118,
         'Inyo': 18,
         'Kern': 370,
         'Kings': 88,
         'Lake': 87,
         'Lassen': 25,
         'Los Angeles': 5824,
         'San Bernardino': 831,
         'Orange': 1618,
         'Ventura': 384,
         'Madera': 75,
         'Marin': 166,
         'Mariposa': 18,
         'Mendocino': 79,
         'Merced': 128,
         'Modoc': 8,
         'Mono': 17,
         'Monterey': 202,
         'Napa': 103,
         'Nevada': 96,
         'San Diego': 1609,
         'Placer': 132,
         'Sacramento': 688,
         'Plumas': 33,
         'Riverside': 588,
         'San Benito': 28,
         'San Fr